In [1]:
# !pip install transformers openai torch torchvision Pillow opencv-python google.generativeai timm

In [2]:
# python3 -m venv social
# virtualenv social
# source social/bin/activate
# pip install jupyter
# pip install ipykernel
# python -m ipykernel install --user --name=social --display-name "Python (social)"
# jupyter notebook

In [3]:
from openai import OpenAI
from transformers import pipeline, DetrImageProcessor, DetrForObjectDetection
import google.generativeai as genai
import PIL.Image as Image
import torch
import warnings
warnings.filterwarnings('ignore')

In [4]:
class HuggingFaceModeration:
    def __init__(self):
        # Text Analysis Pipelines
        ''' 
        Text (Toxicity) Classification: Classifying posts or
        comments as appropriate or inappropriate based on predefined
        categories (e.g., hate speech, spam, harassment). 
        '''
        self.toxicClassifier = pipeline("text-classification", model = "unitary/toxic-bert", device = 0)
        
        '''
        Sentiment Analysis: Analyzing the sentiment of user
        comments to identify negative or harmful sentiments.
        '''
        self.sentimentClassifier = pipeline("sentiment-analysis", model = "cardiffnlp/twitter-roberta-base-sentiment-latest", device = 0)
        
        '''
        Named Entity Recognition (NER): Identifying and categorizing
        entities in text, such as people, organizations, or locations,
        to flag potential privacy violations or inappropriate mentions.
        '''
        self.namedEntityRecognition = pipeline("ner", model = "dbmdz/bert-large-cased-finetuned-conll03-english", device = 0)

        '''
        Text Generation: Generating responses to comments or
        creating auto-replies for moderation actions.
        '''
        self.textGenerator = pipeline("text-generation", model="gpt2", device = 0)

        '''
        Zero-Shot Classification: Classifying text into custom categories
        without needing to train a new model for each category.
        '''
        self.zeroShotClassifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device = 0)
        
        # Image Analysis Pipelines
        '''
        Image Classification: Moderating images uploaded to social media
        platforms by identifying inappropriate content (e.g., violence, nudity).
        '''
        self.imageClassifier = pipeline("image-classification", model="google/vit-base-patch16-224", device = 0)

        '''
        Image Classification (Not Safe for Work): Moderating images uploaded to social media
        which are not safe for work environments, etc.
        by identifying inappropriate content (e.g., violence, nudity).
        '''
        self.nsfwClassifier = pipeline("image-classification", model="Falconsai/nsfw_image_detection", device = 0)
        
        '''
        Image Segmentation: Analyzing specific parts of an image to detect unwanted
        content or to focus on specific areas in user-uploaded images.
        '''
        self.segmentationPipeline = pipeline("image-segmentation", model="facebook/detr-resnet-50", device = 0)

        # Audio Analysis Pipeline
        # Video Analysis Pipeline
    
    def checkToxicity(self, text):
        result = self.toxicClassifier(text)
        label = result[0]['label']
        score = result[0]['score']
        return f"Given text is {label} with score of {round(score, 2)}"

    def analyzeSentiment(self, text):
        result = self.sentimentClassifier(text)
        label = result[0]['label']
        score = result[0]['score']
        return f"Given text shows {label} sentiment with score of {round(score, 2)}"

    def namedEntityRecognizer(self, text):
        result = self.namedEntityRecognition(text)
        formattedResult = []
        for item in result:
            entity = f"Detected {item["word"]} as {item["entity"]} with score of {round(item["score"],2)}"
            formattedResult.append(entity)
        return "\n".join(formattedResult)

    def textGeneration(self, text):
        result = self.textGenerator(text, max_length = 100)
        return result[0]["generated_text"]

    def zeroShotClassification(self, text):
        labels = ["Positive", "Negative", "Neutral"]
        result = self.zeroShotClassifier(text, candidate_labels = labels)
        scores = result["scores"]
        return f"Positive Score: {round(scores[0],2)}, Negative Score: {round(scores[1], 2)} and Neutral Score: {round(scores[2],2)}"
            
    def imageClassification(self, imagePath):
        result = self.imageClassifier(imagePath)
        formattedResult = []
        for item in result:
            object = f"Detected {item["label"]} with score of {round(item["score"], 2)}"
            formattedResult.append(object)
        return "\n".join(formattedResult)

    def detectNSFWContent(self, imagePath):
        image = Image.open(imagePath)
        result = self.nsfwClassifier(image)
        return any(label['label'] == 'NSFW' for label in result)

    def imageSegmentation(self, imagePath):
        image = Image.open(imagePath)

        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

        
        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)

        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.8)[0]

        formattedResult = []
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            box = [round(i, 2) for i in box.tolist()]
            formattedString = (
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
            )
            formattedResult.append(formattedString)
        return "\n".join(formattedResult)

In [5]:
class OpenAIModeration:
    def __init__(self, openAiApiKey):
        self.client = OpenAI(api_key = openAiApiKey)

    def generateSummary(self, text):
        response = self.client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": "You are a amazing text summarization assistant."}, 
                {"role": "user", "content": f"Summarize the following content:\n\n{text}"}
            ],
            max_tokens = 50
        )
        return response.choices[0].message.content

    def detectLanguage(self, text):
        response = self.client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": "You are a highly accurate language detecting assistant."}, 
                {"role": "user", "content": f"Detect the language of the following content:\n\n{text}"}
            ],
            max_tokens = 10
        )
        return response.choices[0].message.content

    def classifyText(self, text):
        response = self.client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": "You are a helpful content classifier assistant."}, 
                {"role": "user", "content": f"Classify the following content into categories like Politics, Entertainment, Science, etc. :\n\n{text}"}
            ],
            max_tokens = 30
        )
        return response.choices[0].message.content

    def decisionForToxicity(self, text):
        response = self.client.chat.completions.create(
            model = "gpt-4o",
            messages = [
                {"role": "system", "content": "You are an intelligent decision maker and you need to decide if the text is toxic or not."},
                {"role": "user", "content": f"Make a decision whether the following text is safe for social media and give reasons for the same: \n\n{text}"}
            ],
            max_tokens = 100
        )
        return response.choices[0].message.content

In [6]:
class GeminiModeration:
    def __init__(self, geminiApiKey):
        genai.configure(api_key = geminiApiKey)
        self.generativeModel = genai.GenerativeModel("gemini-1.5-flash")

    def detectObjectInImage(self, imagePath):
        image = Image.open(imagePath)
        prompt = "Detect all the various objects in the given image."
        response = self.generativeModel.generate_content([prompt, image])
        return response.text

    def checkImageQuality(self, imagePath):
        image = Image.open(imagePath)
        prompt = "Check the image quality."
        response = self.generativeModel.generate_content([prompt, image])
        return response.text
        
    def checkProfanityInImage(self, imagePath):
        image = Image.open(imagePath)
        prompt = "Check if there's any profanity in the image."
        response = self.generativeModel.generate_content([prompt, image])
        return response.text

In [7]:
class ContentModerator:
    def __init__(self, openAiApiKey, geminiApiKey):
        self.huggingFaceModeration = HuggingFaceModeration()
        self.openAiModeration = OpenAIModeration(openAiApiKey)
        self.geminiModeration = GeminiModeration(geminiApiKey)

    def moderateTextContent(self, text):
        report = {}
        
        # Hugging Face Moderation Tasks
        report['Toxicity'] = self.huggingFaceModeration.checkToxicity(text)
        report['Sentiment'] = self.huggingFaceModeration.analyzeSentiment(text)
        report['Named Entity Recognition'] = self.huggingFaceModeration.namedEntityRecognizer(text)
        report['Text Generation'] = self.huggingFaceModeration.textGeneration(text)
        report['Zero Shot Classification'] = self.huggingFaceModeration.zeroShotClassification(text)

        # OpenAI Moderation Tasks
        report['Summary'] = self.openAiModeration.generateSummary(text)
        report['Language'] = self.openAiModeration.detectLanguage(text)
        report['Category'] = self.openAiModeration.classifyText(text)
        report['Decision'] = self.openAiModeration.decisionForToxicity(text)

        return report

    def moderateImageContent(self, imagePath):
        report = {}

        # Hugging Face Image Moderation Tasks
        report['Image Classification'] = self.huggingFaceModeration.imageClassification(imagePath)
        report['Contains NSFW'] = self.huggingFaceModeration.detectNSFWContent(imagePath)
        report['Image Segmentation'] = self.huggingFaceModeration.imageSegmentation(imagePath)

        # Gemini Image Moderation Tasks
        report['Detected Objects'] = self.geminiModeration.detectObjectInImage(imagePath)
        report['Image Quality'] = self.geminiModeration.checkImageQuality(imagePath)
        report['Image Profanity'] = self.geminiModeration.checkProfanityInImage(imagePath)

        return report

    def moderateContent(self, text = None, imagePath = None):
        fullReport = {}
        # If text is provided, perform text moderation
        if text:
            fullReport['textModeration'] = self.moderateTextContent(text)
        # If image is provided, perform image moderation
        if imagePath:
            fullReport['imageModeration'] = self.moderateImageContent(imagePath)

        return fullReport

In [8]:
def setupContentModerator():
    openAiApiKey = ""
    geminiApiKey = ""
    
    moderator = ContentModerator(openAiApiKey, geminiApiKey)
    # moderationReport = moderator.moderateContent(text = textContent, imagePath = imageContentPath)
    return moderator

def printTextModerationReport(report):
    for key, value in report.items():
        print(f"{key} => {value}\n\n")

def printImageModerationReport(report):
    for key, value in report.items():
        print(f"{key} => {value}\n\n")

In [9]:
def main():
    moderator = setupContentModerator()

    while True:
        print("\n--- Social Media Content Moderator Menu ---")
        print("1. Moderate Text Content")
        print("2. Moderate Image Content")
        print("3. Moderate Both Text and Image Content")
        print("4. Find Toxicity Score of Text Content")
        print("5. Find the Sentiment of the Text Content")
        print("6. Identify the Entities from the Text Content")
        print("7. Detect Objects in the Image")
        print("8. Profanity in the Image")
        print("9. Decision whether the Text is Safe for Social Media")
        print("10. Exit")

        choice = input("Select an option (1 - 10): ")
        
        if choice == '1':
            textContent = input("Enter the text: ")
            textModerationReport = moderator.moderateTextContent(textContent)
            printTextModerationReport(textModerationReport)
        elif choice == '2':
            imagePath = input("Enter the image path: ")
            imageModerationReport = moderator.moderateImageContent(imagePath)
            printImageModerationReport(imageModerationReport)
        elif choice == '3':
            textContent = input("Enter the text:")
            imagePath = input("Enter the image path: ")
            moderationReport = moderator.moderateContent(textContent, imagePath)
            printTextModerationReport(moderationReport['textModeration'])
            printImageModerationReport(moderationReport['imageModeration'])
        elif choice == '4':
            textContent = input("Enter the text:")
            result = moderator.huggingFaceModeration.checkToxicity(textContent)
            print(f"Toxicity: {result}")
        elif choice == '5':
            textContent = input("Enter the text:")
            result = moderator.huggingFaceModeration.zeroShotClassification(textContent)
            print(f"Sentiment: {result}")
        elif choice == '6':
            result = moderator.huggingFaceModeration.namedEntityRecognizer(textContent)
            print(f"Named Entity Recognition: {result}")
        elif choice == '7':
            imagePath = input("Enter the image path: ")
            result = moderator.geminiModeration.detectObjectInImage(imagePath)
            print(f"Detected Objects: {result}")
        elif choice == '8':
            imagePath = input("Enter the image path: ")
            result = moderator.geminiModeration.checkProfanityInImage(imagePath)
            print(f"Profanity: {result}")
        elif choice == '9':
            textContent = input("Enter the text:")
            result = moderator.openAiModeration.decisionForToxicity(textContent)
            print(f"Decision for Toxicity: {result}")
        elif choice == '10':
            print("Exiting the content moderation tool. Goodbye!")
            break
        else:
            print("Wrong choice, select a valid option from the content moderation tool!")

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch


--- Social Media Content Moderator Menu ---
1. Moderate Text Content
2. Moderate Image Content
3. Moderate Both Text and Image Content
4. Find Toxicity Score of Text Content
5. Find the Sentiment of the Text Content
6. Identify the Entities from the Text Content
7. Detect Objects in the Image
8. Profanity in the Image
9. Decision whether the Text is Safe for Social Media
10. Exit


Select an option (1 - 10):  1
Enter the text:  Woke up feeling grateful for all the little things in life. 😊 Here’s to a great day ahead for everyone! Sending positive vibes to anyone who needs it today! 🌟 You’re doing amazing, keep going! I am having fun here in Paris, You should definitely see the Eiffel Tower


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Toxicity => Given text is toxic with score of 0.0


Sentiment => Given text shows positive sentiment with score of 0.99


Named Entity Recognition => Detected Paris as I-LOC with score of 1.0
Detected E as I-MISC with score of 0.5600000023841858
Detected ##iff as I-LOC with score of 0.6600000262260437
Detected ##el as I-LOC with score of 0.6499999761581421
Detected Tower as I-LOC with score of 0.8799999952316284


Text Generation => Woke up feeling grateful for all the little things in life. 😊 Here’s to a great day ahead for everyone! Sending positive vibes to anyone who needs it today! 🌟 You’re doing amazing, keep going! I am having fun here in Paris, You should definitely see the Eiffel Tower with everyone and thank you ❤️ ♥ Thank you for everyone's support and to all my friends from Paris! ❤️ My name is Kait


Zero Shot Classification => Positive Score: 0.96, Negative Score: 0.03 and Neutral Score: 0.01


Summary => The author expresses gratitude for life's small joys, sends positiv

...

Sentiment: Positive Score: 0.93, Negative Score: 0.04 and Neutral Score: 0.03

--- Social Media Content Moderator Menu ---
1. Moderate Text Content
2. Moderate Image Content
3. Moderate Both Text and Image Content
4. Find Toxicity Score of Text Content
5. Find the Sentiment of the Text Content
6. Identify the Entities from the Text Content
7. Detect Objects in the Image
8. Profanity in the Image
9. Decision whether the Text is Safe for Social Media
10. Exit


Select an option (1 - 10):  4
Enter the text: "Everything you post is total garbage. Stop polluting my feed." "I hate seeing people like you on my timeline. Just disappear." "One day, someone’s going to make sure you pay for everything you’ve done." "You’re lucky people don’t come after you. Keep talking and see what happens." "That community is full of idiots. They should just stay out of our way." "Why do those people always act like this? It’s so typical of them."


Toxicity: Given text is toxic with score of 0.96

--- Social Media Content Moderator Menu ---
1. Moderate Text Content
2. Moderate Image Content
3. Moderate Both Text and Image Content
4. Find Toxicity Score of Text Content
5. Find the Sentiment of the Text Content
6. Identify the Entities from the Text Content
7. Detect Objects in the Image
8. Profanity in the Image
9. Decision whether the Text is Safe for Social Media
10. Exit


Select an option (1 - 10):  10


Exiting the content moderation tool. Goodbye!
